# Batch deployments with a custom output

In [2]:
from azure.ai.ml import MLClient, Input
from azure.ai.ml.entities import (
    BatchEndpoint,
    BatchDeployment,
    Model,
    AmlCompute,
    Data,
    BatchRetrySettings,
    CodeConfiguration,
    Environment,
)
from azure.ai.ml.constants import AssetTypes, BatchDeploymentOutputAction
from azure.identity import DefaultAzureCredential

In [4]:
subscription_id = "<subscription>"
resource_group = "<resource-group>"
workspace = "<workspace>"

ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

Let's verify if the model we want to deploy, `heart-classifier`, is registered in the model registry. If not, we will register it from a local version we have in the repository:

In [5]:
model_name = "heart-classifier"
model_local_path = "heart-classifier-mlflow/model"

In [7]:
if not any(filter(lambda m: m.name == model_name, ml_client.models.list())):
    print(f"Model {model_name} is not registered. Creating...")
    model = ml_client.models.create_or_update(
        Model(name=model_name, path=model_local_path, type=AssetTypes.MLFLOW_MODEL)
    )

Let's get the model:

In [8]:
model = ml_client.models.get(name=model_name, label="latest")

First, let's create the endpoint that is going to host the batch deployments. Remember that each endpoint can host multiple deployments at any time, however, only one of them is the default one:

In [9]:
endpoint = BatchEndpoint(
    name="heart-classifier-batch",
    description="A heart condition classifier for batch inference",
)

In [10]:
ml_client.batch_endpoints.begin_create_or_update(endpoint)

Batch endpoints can run on any Azure ML compute that already exists in the workspace. That means that multiple batch deployments can share the same compute infrastructure. In this example, we are going to work on an AzureML compute cluster called `cpu-cluster`. Let's verify the compute exists on the workspace or create it otherwise.

In [11]:
compute_name = "cpu-cluster"
if not any(filter(lambda m: m.name == compute_name, ml_client.compute.list())):
    print(f"Compute {compute_name} is not created. Creating...")
    compute_cluster = AmlCompute(
        name=compute_name, description="amlcompute", min_instances=0, max_instances=5
    )
    ml_client.begin_create_or_update(compute_cluster)

Compute may take time to be created. Let's wait for it:

In [12]:
from time import sleep

print("Waiting for compute", end="")
while ml_client.compute.get(name=compute_name).provisioning_state == "Creating":
    sleep(1)
    print(".", end="")

print(" [DONE]")

Waiting for compute [DONE]


Authoring an scoring script that can write to the output folder:

In [51]:
%%writefile heart-classifier-mlflow/code/batch_driver_parquet.py

import os
import mlflow
import pandas as pd
from pathlib import Path

def init():
    global model
    global output_path

    # AZUREML_MODEL_DIR is an environment variable created during deployment
    # It is the path to the model folder
    # Please provide your model's folder name if there's one:
    model_path = os.path.join(os.environ["AZUREML_MODEL_DIR"], "model")
    output_path = os.environ['AZUREML_BI_OUTPUT_PATH']
    model = mlflow.pyfunc.load_model(model_path)

def run(mini_batch):
    for file_path in mini_batch:        
        data = pd.read_csv(file_path)
        pred = model.predict(data)
        
        data['prediction'] = pred
        
        output_file_name = Path(file_path).stem
        output_file_path = os.path.join(output_path, output_file_name + '.parquet')
        data.to_parquet(output_file_path)
    
    return mini_batch


Overwriting heart-classifier-mlflow/code/batch_driver_parquet.py


Let's create a deployment under the given endpoint.

In [52]:
environment = Environment(
    conda_file="./heart-classifier-mlflow/environment/conda.yml",
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:latest",
)

In [53]:
deployment = BatchDeployment(
    name="classifier-xgboost-parquet",
    description="A heart condition classifier based on XGBoost",
    endpoint_name=endpoint.name,
    model=model,
    environment=environment,
    code_configuration=CodeConfiguration(
        code="./heart-classifier-mlflow/code/",
        scoring_script="batch_driver_parquet.py",
    ),
    compute=compute_name,
    instance_count=2,
    max_concurrency_per_instance=2,
    mini_batch_size=2,
    output_action=BatchDeploymentOutputAction.SUMMARY_ONLY,
    retry_settings=BatchRetrySettings(max_retries=3, timeout=300),
    logging_level="info",
)

In [54]:
ml_client.batch_deployments.begin_create_or_update(deployment)

Uploading code (0.0 MBs): 100%|██████████| 2042/2042 [00:00<00:00, 20095.33it/s]




## Testing the endpoint

Once the deployment is created, it is ready to recieve jobs. Let's first register a data asset so we can run the job against it. This data asset is a folder containing multiple CSV files that we want to process in parallel using the batch endpoint we just created.

In [46]:
data_path = "heart-classifier-mlflow/dataset/"
dataset_name = "heart-dataset-unlabeled"

heart_dataset_unlabeled = Data(
    path=data_path,
    type=AssetTypes.URI_FOLDER,
    description="An unlabeled dataset for heart classification",
    name=dataset_name,
)

ml_client.data.create_or_update(heart_dataset_unlabeled)

Uploading dataset (0.01 MBs): 100%|██████████| 12833/12833 [00:00<00:00, 156020.93it/s]




Data({'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_folder', 'is_anonymous': False, 'auto_increment_version': False, 'name': 'heart-dataset-unlabeled', 'description': 'An unlabeled dataset for heart classification', 'tags': {}, 'properties': {}, 'id': '/subscriptions/18522758-626e-4d88-92ac-dc9c7a5c26d4/resourceGroups/Analytics.Aml.Experiments.Workspaces/providers/Microsoft.MachineLearningServices/workspaces/aa-ml-aml-workspace/data/heart-dataset-unlabeled/versions/7', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/santiagxf-cpu/code/Users/fasantia/azureml-examples/sdk/python/endpoints/batch', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7fc56fb71be0>, 'serialize': <msrest.serialization.Serializer object at 0x7fc56fb717f0>, 'version': '7', 'latest_version': None, 'path': 'azureml://subscriptions/18522758-626e-4d88-92ac-dc9c7a5c26d4/resourcegroups/Analytics.Aml.Ex

In [47]:
heart_dataset_unlabeled = ml_client.data.get(name=dataset_name, label="latest")

Let's use this data as an input for the job:

In [48]:
input = Input(type=AssetTypes.URI_FOLDER, path=heart_dataset_unlabeled.id)

In [55]:
job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint.name, deployment_name=deployment.name, input=input
)

You can use the returned job object to check the status of the job:

In [56]:
ml_client.jobs.get(job.name)

Experiment,Name,Type,Status,Details Page
heart-classifier-batch,5e4d176f-7024-4fa2-9d25-92176b7ed573,pipeline,Preparing,Link to Azure Machine Learning studio


## Exploring the results

We can download the results from the job by downloading the output with name `score`:

In [57]:
ml_client.jobs.download(name=job.name, download_path=".", output_name="score")

In [61]:
import pandas as pd
import glob

output_files = glob.glob("named-outputs/score/*.parquet")
score = pd.concat((pd.read_parquet(f) for f in output_files))

In [62]:
score

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,prediction
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,52,1,1,118,186,0,2,190,0,0.0,2,0,fixed,0
99,43,0,4,132,341,1,2,136,1,3.0,2,0,reversible,1
0,65,1,4,135,254,0,2,127,0,2.8,2,1,reversible,1
1,48,1,4,130,256,1,2,150,1,0.0,1,2,reversible,1
